In [0]:
USE CATALOG users;
USE users.yukiteru_koide;

CREATE OR REPLACE TABLE users.yukiteru_koide.signin_events_silver AS
WITH base AS (
  SELECT
    id,
    to_timestamp(createdDateTime) AS created_ts,
    to_date(to_timestamp(createdDateTime)) AS event_date,
    hour(to_timestamp(createdDateTime)) AS event_hour,
    userPrincipalName,
    userId,
    appDisplayName,
    appId,
    ipAddress,
    clientAppUsed,
    CASE lower(isInteractive) WHEN 'true' THEN true WHEN 'false' THEN false ELSE NULL END AS isInteractive,
    authenticationRequirement,
    conditionalAccessStatus,
    deviceOperatingSystem,
    browser,
    locationCountryOrRegion,
    locationCity,
    CAST(statusErrorCode AS INT) AS statusErrorCode,
    statusFailureReason,
    statusAdditionalDetails,
    (statusErrorCode = 0) AS is_success,
    (statusErrorCode <> 0) AS is_failure,
    (authenticationRequirement = 'multiFactorAuthentication') AS is_mfa,
    (conditionalAccessStatus = 'failure') AS ca_failed,
    (hour(to_timestamp(createdDateTime)) < 8 OR hour(to_timestamp(createdDateTime)) >= 20) AS is_off_hours
  FROM users.yukiteru_koide.signin_logs_bronze
),
travel AS (
  SELECT
    *,
    LAG(locationCountryOrRegion) OVER (PARTITION BY userId ORDER BY created_ts) AS prev_country,
    LAG(locationCity) OVER (PARTITION BY userId ORDER BY created_ts) AS prev_city,
    LAG(created_ts) OVER (PARTITION BY userId ORDER BY created_ts) AS prev_ts
  FROM base
)
SELECT
  *,
  CASE
    WHEN prev_country IS NOT NULL
     AND (locationCountryOrRegion <> prev_country OR locationCity <> prev_city)
     AND (unix_timestamp(created_ts) - unix_timestamp(prev_ts)) <= 3600
    THEN true ELSE false
  END AS impossible_travel
FROM travel;

OPTIMIZE users.yukiteru_koide.signin_events_silver;
